In [ ]:
from bs4 import BeautifulSoup as bs
from urllib.request import Request, urlopen
import pandas as pd
import time

countries = {}
datalist = []

def get_countries():
    homepage = 'http://www.swiftcodelist.com/countries.html'
    req = Request(homepage, headers={'User-Agent': 'Mozilla/5.0'})
    countrypage = urlopen(req).read()
    soup = bs(countrypage, "html.parser")
    # get list of countries and its url 
    for c in soup.select('a[href^=http://www.swiftcodelist.com/country/]'):
        countries[c.string] = c.get('href')

def get_swift_code(country, url):
    time.sleep(1)
    c_page = urlopen(Request(url,headers={'User-Agent': 'Mozilla/5.0'})).read()
    c_soup =bs(c_page, "html.parser")
    # skip 1st row 
    rows = c_soup.table.find_all('tr')
    iterr = iter(rows)
    next(iterr)
    # rows
    for row in iterr:
        td = row.find_all('td')
        # columns
        bank_name = td[1].a.string
        branch_name = td[2].string
        city = td[3].string
        swift = td[4].a.string
        datalist.append([bank_name, branch_name,city,country,swift])

def get_page_range(url):
    page = urlopen(Request(url,headers={'User-Agent': 'Mozilla/5.0'})).read()
    soup =bs(page, "html.parser")
    page_range = soup.find('div',class_='pageNav').text
    pages = [int(s) for s in page_range.split() if s.isdigit()]
    return pages[1]

def scrape_to_df():
    # get swift code for each countries
    countries = {'India': 'http://www.swiftcodelist.com/country/india.html'}
    for k,v in countries.items():
        #for initial page
        get_swift_code(k,v)
        print(v)
        last_page = get_page_range(v)
        #for subsequent pages
        for i in range(2,last_page+1):
            url = v[:-5] + '-' + str(i) + '.html'
            print(url)
            get_swift_code(k,url)

In [ ]:
scrape_to_df()


In [ ]:
headers = ['BankName','BranchName','City','Country','SWIFTCode']
df = pd.DataFrame(datalist,columns=headers)

In [ ]:
df